# Regression
## 지도학습 (타겟변수가 존재)

In [4]:
import pymysql.cursors
import pandas as pd

In [5]:
conn = pymysql.connect(host='localhost', user='me', 
                       password='1234', db='tip', charset='utf8',
                       autocommit=True, cursorclass=pymysql.cursors.DictCursor)
try:

   with conn.cursor() as curs:
      sql = "select * from tips;"
      curs.execute(sql)
      rs = curs.fetchall()

      # DB에서 받아온 값을 DataFrame에 넣음

      df = pd.DataFrame(rs)    
#       df.to_csv('query.csv')

finally:

   conn.close()

In [6]:
df["sex"] = df["sex"].replace({"Female" : 0, "Male" : 1})
# tips['sex'].replace(~~~~, inplace = True) 해당 변경값을 바로 변수에 투입
# 
df["smoker"] = df["smoker"].replace({"No" : 0, "Yes" : 1})
df["day"] = df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3})
df["time"] = df["time"].replace({"Lunch" : 0, "Dinner" : 1})


In [7]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size
count,249.000000,249.000000,247.000000,249.000000,249.000000,248.000000,248.000000
mean,19.622972,2.968795,0.643725,0.373494,1.751004,0.725806,2.572581
std,8.893766,1.385573,0.479870,0.484706,1.158006,0.447009,0.945249
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,13.160000,2.000000,0.000000,0.000000,1.000000,0.000000,2.000000
50%,17.510000,2.750000,1.000000,0.000000,2.000000,1.000000,2.000000
75%,24.060000,3.510000,1.000000,1.000000,3.000000,1.000000,3.000000
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000


In [8]:
df.isnull().sum()

total_bill    0
tip           0
sex           2
smoker        0
day           0
time          1
size          1
dtype: int64

In [9]:
from numpy import isnan
from sklearn.impute import SimpleImputer

In [10]:
y = df['total_bill']
x = df.drop('total_bill',axis =1)

In [11]:
y

0      16.99
1      10.34
2      21.01
3      23.68
4      24.59
       ...  
244    16.99
245    10.34
246    10.34
247    10.34
248    10.34
Name: total_bill, Length: 249, dtype: float64

In [12]:
x

,tip,sex,smoker,day,time,size
0,1.01,0.0,0,3,1.0,2.0
1,1.66,1.0,0,3,1.0,3.0
2,3.50,1.0,0,3,1.0,3.0
3,3.31,1.0,0,3,1.0,2.0
4,3.61,0.0,0,3,1.0,4.0
...,...,...,...,...,...,...
244,1.01,0.0,0,3,1.0,2.0
245,1.66,NaN,0,3,1.0,3.0
246,1.66,NaN,0,3,1.0,3.0
247,1.66,1.0,0,3,NaN,3.0


In [13]:
data = x.values
y = y.values

In [14]:
sum(isnan(data).flatten())

4

In [15]:
sum(isnan(y).flatten())

0

In [16]:
imputer = SimpleImputer(strategy = 'median')
imputer.fit(data)
data_trans = imputer.transform(data)

## regression 시작

In [17]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [18]:
fs = SelectKBest(score_func=f_regression, k=4)
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)
print(X_selected.shape)

(249, 4)


In [21]:
df2 = pd.DataFrame(X_selected)

In [22]:
df2

,0,1,2,3
0,1.01,3.0,1.0,2.0
1,1.66,3.0,1.0,3.0
2,3.50,3.0,1.0,3.0
3,3.31,3.0,1.0,2.0
4,3.61,3.0,1.0,4.0
...,...,...,...,...
244,1.01,3.0,1.0,2.0
245,1.66,3.0,1.0,3.0
246,1.66,3.0,1.0,3.0
247,1.66,3.0,1.0,3.0
